In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.simplefilter('ignore')

In [2]:
data = pd.read_csv(r'C:/Users/uzmap/Documents/GitHub/ForEx/data_For_prophet.csv',index_col='Unnamed: 0')
data.head()

,Unnamed: 0.1,Date,Open_price,Day_high,Day_low,Closing_price,Currency Pair,Trend_Open_price,Seasonal_Open_price,Residual_Open_price,...,Seasonal_Day_low,Residual_Day_low,RSI,MACD,MACD_Signal,BB_Upper,BB_Lower,day_of_week,month,is_weekend
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
0.000000,0,2014-11-07,0.386891,0.384657,0.389693,0.386954,USD/INR,0.394461,0.522182,0.60751,...,0.473615,0.538767,64.491363,0.000000,0.000000,0.391812,0.38654,4,11,0
0.000055,1,2014-11-10,0.387590,0.384752,0.389693,0.387558,USD/INR,0.394461,0.557718,0.60751,...,0.460915,0.538767,64.491363,0.000048,0.000010,0.391812,0.38654,0,11,0
0.000109,2,2014-11-11,0.387781,0.384248,0.390750,0.387641,USD/INR,0.394461,0.533733,0.60751,...,0.475153,0.538767,64.491363,0.000092,0.000026,0.391812,0.38654,1,11,0
0.000164,3,2014-11-12,0.387641,0.384280,0.389757,0.386897,USD/INR,0.394461,0.541190,0.60751,...,0.422273,0.538767,64.491363,0.000066,0.000034,0.391812,0.38654,2,11,0
0.000219,4,2014-11-13,0.386752,0.384676,0.389757,0.388003,USD/INR,0.394461,0.534029,0.60751,...,0.456504,0.538767,64.491363,0.000133,0.000054,0.391812,0.38654,3,11,0


Prophet

In [19]:
import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score

# Convert Date column to datetime (if necessary)
data['Date'] = pd.to_datetime(data['Date'])

# List of unique currency pairs
currency_pairs = data['Currency Pair'].unique()

# Dictionary to store evaluation metrics for each currency pair
results = {}

# Loop over each currency pair
for pair in currency_pairs:
    print(f"Training model for currency pair: {pair}")
    
    # Filter data for the current currency pair
    pair_data = data[data['Currency Pair'] == pair]
    
    # Sort by date to ensure proper time series order
    pair_data = pair_data.sort_values('Date')
    
    # Prepare data for Prophet (needs columns 'ds' and 'y')
    df_prophet = pair_data[['Date', 'Closing_price']].rename(columns={'Date': 'ds', 'Closing_price': 'y'})
    
    # Add the regressors to the data (these are assumed to be already in the dataset)
    regressors = [
        'RSI', 'MACD', 'MACD_Signal', 'BB_Upper', 'BB_Lower', 
        'is_weekend', 'Seasonal_Open_price'
    ]
    
    for regressor in regressors:
        df_prophet[regressor] = pair_data[regressor]
    
    # Split data into train and test sets (80-20 split)
    split_idx = int(len(df_prophet) * 0.8)
    train_data = df_prophet.iloc[:split_idx]
    test_data = df_prophet.iloc[split_idx:]
    
    # Initialize the Prophet model and add regressors
    model = Prophet(daily_seasonality=True, yearly_seasonality=True, weekly_seasonality=True)
    
    # Add each regressor to the model
    for regressor in regressors:
        model.add_regressor(regressor)
    
    # Fit the model
    model.fit(train_data)

    # Make future DataFrame for predictions (including the regressors)
    future = model.make_future_dataframe(periods=len(test_data), freq='D')
    
    # Add the regressors to the future DataFrame
    for regressor in regressors:
        future[regressor] = pair_data[regressor].iloc[-len(future):].values
    
    # Forecast using Prophet
    forecast = model.predict(future)
    
    # Extract the forecasted values for the training and test periods
    y_pred_train = forecast['yhat'].iloc[:split_idx].values
    y_train = train_data['y'].values
    
    y_pred_test = forecast['yhat'].iloc[split_idx:].values
    y_test = test_data['y'].values

    # Calculate evaluation metrics for training data
    mae_train = mean_absolute_error(y_train, y_pred_train)
    mape_train = mean_absolute_percentage_error(y_train, y_pred_train)
    r2_train = r2_score(y_train, y_pred_train)
    
    # Calculate evaluation metrics for test data
    mae_test = mean_absolute_error(y_test, y_pred_test)
    mape_test = mean_absolute_percentage_error(y_test, y_pred_test)
    r2_test = r2_score(y_test, y_pred_test)

    # Calculate difference in R² between training and test to check for overfitting
    r2_difference = r2_train - r2_test
    
    # Store the results and predictions for this currency pair
    results[pair] = {
        'MAE_train': mae_train,
        'MAPE_train': mape_train,
        'R^2_train': r2_train,
        'MAE_test': mae_test,
        'MAPE_test': mape_test,
        'R^2_test': r2_test,
        'R^2_difference': r2_difference,
        'y_true_test': y_test,       # True values for the test set
        'y_pred_test': y_pred_test    # Predicted values for the test set
    }

# Display final results and predictions for each currency pair
print("\nFinal Results for Each Currency Pair:")
for pair, metrics in results.items():
    print(f"{pair} - Train: MAE: {metrics['MAE_train']}, MAPE: {metrics['MAPE_train']}, R^2: {metrics['R^2_train']}")
    print(f"Test: MAE: {metrics['MAE_test']}, MAPE: {metrics['MAPE_test']}, R^2: {metrics['R^2_test']}")
    print(f"Difference in R² (Train - Test): {metrics['R^2_difference']}")
    print(f"True values (sample): {metrics['y_true_test'][:5]}")
    print(f"Predicted values (sample): {metrics['y_pred_test'][:5]}\n")


Training model for currency pair: USD/INR


17:40:09 - cmdstanpy - INFO - Chain [1] start processing
17:40:10 - cmdstanpy - INFO - Chain [1] done processing


Training model for currency pair: EUR/USD


17:40:11 - cmdstanpy - INFO - Chain [1] start processing
17:40:12 - cmdstanpy - INFO - Chain [1] done processing


Training model for currency pair: GBP/USD


...

Training model for currency pair: JPY/INR


17:40:18 - cmdstanpy - INFO - Chain [1] done processing


Training model for currency pair: GBP/INR


17:40:18 - cmdstanpy - INFO - Chain [1] start processing
17:40:19 - cmdstanpy - INFO - Chain [1] done processing



Final Results for Each Currency Pair:
USD/INR - Train: MAE: 0.000689587683939987, MAPE: 0.0015613995472382107, R^2: 0.9991214520708618
Test: MAE: 0.002532717698598882, MAPE: 0.004838223636468014, R^2: 0.3250390477967763
Difference in R² (Train - Test): 0.6740824042740855
True values (sample): [0.51128423 0.51189428 0.51432814 0.51519238 0.51543386]
Predicted values (sample): [0.50290394 0.50348122 0.51263309 0.5136242  0.51461454]

EUR/USD - Train: MAE: 1.9750040484407092e-05, MAPE: 0.005131994942927224, R^2: 0.9893266914957639
Test: MAE: 5.074404420918985e-05, MAPE: 0.013904686268811847, R^2: 0.6257269309711652
Difference in R² (Train - Test): 0.36359976052459875
True values (sample): [0.00335682 0.00338039 0.00335923 0.00333966 0.00328552]
Predicted values (sample): [0.00332713 0.00336176 0.00337825 0.0034516  0.00344386]

GBP/USD - Train: MAE: 3.586474522425106e-05, MAPE: 0.006813605727640526, R^2: 0.9804772712521642
Test: MAE: 2.680812377791883e-05, MAPE: 0.005659609397651212, R^2

Prophet with XGboost

In [20]:
import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score
import xgboost as xgb

# Convert Date column to datetime (if necessary)
data['Date'] = pd.to_datetime(data['Date'])

# List of unique currency pairs
currency_pairs = data['Currency Pair'].unique()

# Dictionary to store evaluation metrics for each currency pair
results = {}

# Loop over each currency pair
for pair in currency_pairs:
    print(f"Training model for currency pair: {pair}")
    
    # Filter data for the current currency pair
    pair_data = data[data['Currency Pair'] == pair]
    
    # Sort by date to ensure proper time series order
    pair_data = pair_data.sort_values('Date')
    
    # Prepare data for Prophet (needs columns 'ds' and 'y')
    df_prophet = pair_data[['Date', 'Closing_price']].rename(columns={'Date': 'ds', 'Closing_price': 'y'})
    
    # Add the regressors to the data (these are assumed to be already in the dataset)
    regressors = [
        'RSI', 'MACD', 'MACD_Signal', 'BB_Upper', 'BB_Lower', 
        'is_weekend', 'Seasonal_Open_price'
    ]
    
    for regressor in regressors:
        df_prophet[regressor] = pair_data[regressor]
    
    # Split data into train and test sets (80-20 split)
    split_idx = int(len(df_prophet) * 0.8)
    train_data = df_prophet.iloc[:split_idx]
    test_data = df_prophet.iloc[split_idx:]
    
    # Initialize the Prophet model and add regressors
    model = Prophet(daily_seasonality=True, yearly_seasonality=True, weekly_seasonality=True)
    
    # Add each regressor to the model
    for regressor in regressors:
        model.add_regressor(regressor)
    
    # Fit the Prophet model
    model.fit(train_data)
    
    # Create a DataFrame for training predictions with regressors
    future_train = train_data[['ds']].copy()
    for regressor in regressors:
        future_train[regressor] = train_data[regressor]
    
    # Forecast using Prophet for training data
    forecast_train = model.predict(future_train)
    y_pred_train_prophet = forecast_train['yhat'].values
    y_train = train_data['y'].values
    
    # Make future DataFrame for test predictions (including the regressors)
    future_test = model.make_future_dataframe(periods=len(test_data), freq='D')
    for regressor in regressors:
        future_test[regressor] = pair_data[regressor].iloc[-len(future_test):].values
    
    # Forecast using Prophet for test data
    forecast_test = model.predict(future_test)
    y_pred_test_prophet = forecast_test['yhat'].iloc[split_idx:].values
    y_test = test_data['y'].values
    
    # Calculate residuals for both training and test sets
    residuals_train = y_train - y_pred_train_prophet
    residuals_test = y_test - y_pred_test_prophet
    
    # Prepare XGBoost for residual correction on training data
    xgboost_data_train = train_data[regressors].copy()
    xgboost_data_train['residual'] = residuals_train
    X_train = xgboost_data_train[regressors]
    y_residual_train = xgboost_data_train['residual']
    
    # Prepare XGBoost for residual correction on test data
    xgboost_data_test = test_data[regressors].copy()
    xgboost_data_test['residual'] = residuals_test
    X_test = xgboost_data_test[regressors]
    y_residual_test = xgboost_data_test['residual']
    
    # Initialize the XGBoost model and fit it on the training residuals
    xgboost_model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.05, max_depth=6)
    xgboost_model.fit(X_train, y_residual_train)
    
    # Predict residuals on both training and test sets
    y_pred_residual_train = xgboost_model.predict(X_train)
    y_pred_residual_test = xgboost_model.predict(X_test)
    
    # Combine Prophet and XGBoost predictions for both sets
    y_pred_train_combined = y_pred_train_prophet + y_pred_residual_train
    y_pred_test_combined = y_pred_test_prophet + y_pred_residual_test
    
    # Calculate evaluation metrics for training data
    mae_train = mean_absolute_error(y_train, y_pred_train_combined)
    mape_train = mean_absolute_percentage_error(y_train, y_pred_train_combined)
    r2_train = r2_score(y_train, y_pred_train_combined)
    
    # Calculate evaluation metrics for test data
    mae_test = mean_absolute_error(y_test, y_pred_test_combined)
    mape_test = mean_absolute_percentage_error(y_test, y_pred_test_combined)
    r2_test = r2_score(y_test, y_pred_test_combined)
    
    # Calculate difference in R² between training and test to check for overfitting
    r2_difference = r2_train - r2_test
    
    # Store the results and predictions for this currency pair
    results[pair] = {
        'MAE_train': mae_train,
        'MAPE_train': mape_train,
        'R^2_train': r2_train,
        'MAE_test': mae_test,
        'MAPE_test': mape_test,
        'R^2_test': r2_test,
        'R^2_difference': r2_difference,
        'y_true_test': y_test,         # True values for the test set
        'y_pred_test': y_pred_test_combined  # Predicted values for the test set
    }

# Display final results for each currency pair
print("\nFinal Results for Each Currency Pair:")
for pair, metrics in results.items():
    print(f"{pair} - Train: MAE: {metrics['MAE_train']}, MAPE: {metrics['MAPE_train']}, R^2: {metrics['R^2_train']}")
    print(f"Test: MAE: {metrics['MAE_test']}, MAPE: {metrics['MAPE_test']}, R^2: {metrics['R^2_test']}")
    print(f"Difference in R² (Train - Test): {metrics['R^2_difference']}")
    print(f"True values (sample): {metrics['y_true_test'][:5]}")
    print(f"Predicted values (sample): {metrics['y_pred_test'][:5]}\n")


Training model for currency pair: USD/INR


17:41:24 - cmdstanpy - INFO - Chain [1] start processing
17:41:25 - cmdstanpy - INFO - Chain [1] done processing


Training model for currency pair: EUR/USD


17:41:26 - cmdstanpy - INFO - Chain [1] start processing
17:41:27 - cmdstanpy - INFO - Chain [1] done processing


Training model for currency pair: GBP/USD


...

Training model for currency pair: JPY/INR


17:41:34 - cmdstanpy - INFO - Chain [1] start processing
17:41:35 - cmdstanpy - INFO - Chain [1] done processing


Training model for currency pair: GBP/INR


17:41:37 - cmdstanpy - INFO - Chain [1] start processing
17:41:38 - cmdstanpy - INFO - Chain [1] done processing



Final Results for Each Currency Pair:
USD/INR - Train: MAE: 0.0005673028571944296, MAPE: 0.001291290831304136, R^2: 0.9994246616362678
Test: MAE: 0.0026293276657167403, MAPE: 0.005023193288588003, R^2: 0.20502774044305316
Difference in R² (Train - Test): 0.7943969211932146
True values (sample): [0.51128423 0.51189428 0.51432814 0.51519238 0.51543386]
Predicted values (sample): [0.50181487 0.50251188 0.5117108  0.51265706 0.51365996]

EUR/USD - Train: MAE: 1.9750043401254852e-05, MAPE: 0.005131998054123242, R^2: 0.9893266915749953
Test: MAE: 5.0745888461136485e-05, MAPE: 0.013905189660229501, R^2: 0.6257048750447303
Difference in R² (Train - Test): 0.36362181653026504
True values (sample): [0.00335682 0.00338039 0.00335923 0.00333966 0.00328552]
Predicted values (sample): [0.00332713 0.00336177 0.00337826 0.0034516  0.00344386]

GBP/USD - Train: MAE: 3.586477205745934e-05, MAPE: 0.006813611313640423, R^2: 0.9804772712650194
Test: MAE: 2.680863695017618e-05, MAPE: 0.0056597228181537, R^

ARIMA

In [21]:
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score
import pandas as pd

# Convert Date column to datetime (if necessary)
data['Date'] = pd.to_datetime(data['Date'])

# List of unique currency pairs
currency_pairs = data['Currency Pair'].unique()

# Dictionary to store evaluation metrics for each currency pair
results = {}

# Loop over each currency pair
for pair in currency_pairs:
    print(f"Training ARIMA model for currency pair: {pair}")
    
    # Filter data for the current currency pair
    pair_data = data[data['Currency Pair'] == pair]
    
    # Sort by date to ensure proper time series order
    pair_data = pair_data.sort_values('Date')
    
    # Prepare data for ARIMA (use 'Date' as index, and 'Closing_price' as target variable)
    pair_data.set_index('Date', inplace=True)
    y = pair_data['Closing_price']
    
    # Add any external regressors (additional features), such as RSI, MACD, etc.
    regressors = [
        'RSI', 'MACD', 'MACD_Signal', 'BB_Upper', 'BB_Lower', 
        'is_weekend','Seasonal_Open_price'
    ]
    
    # Include regressors in the ARIMA model (ARIMAX)
    X = pair_data[regressors]

    # Split data into train and test sets (80-20 split)
    split_idx = int(len(pair_data) * 0.8)
    train_y = y[:split_idx]
    test_y = y[split_idx:]
    train_X = X[:split_idx]
    test_X = X[split_idx:]
    
    # Fit ARIMA model with ARIMAX (ARIMA with exogenous variables)
    model = ARIMA(train_y, exog=train_X, order=(5, 1, 0))  # (p, d, q) order of the ARIMA model
    model_fit = model.fit()

    # Make predictions on both the training and test sets
    # For training predictions, use the first and last index of the training data
    y_pred_train = model_fit.predict(start=train_y.index[0], end=train_y.index[-1], exog=train_X)
    
    # For test predictions, use the start and end index of the test data
    y_pred_test = model_fit.predict(start=test_y.index[0], end=test_y.index[-1], exog=test_X)

    # Calculate evaluation metrics for both training and test sets
    mae_train = mean_absolute_error(train_y, y_pred_train)
    mape_train = mean_absolute_percentage_error(train_y, y_pred_train)
    r2_train = r2_score(train_y, y_pred_train)
    
    mae_test = mean_absolute_error(test_y, y_pred_test)
    mape_test = mean_absolute_percentage_error(test_y, y_pred_test)
    r2_test = r2_score(test_y, y_pred_test)
    
    # Calculate the difference in R² between training and test to check for overfitting
    r2_difference = r2_train - r2_test
    
    # Store the results for this currency pair
    results[pair] = {
        'MAE_train': mae_train,
        'MAPE_train': mape_train,
        'R^2_train': r2_train,
        'MAE_test': mae_test,
        'MAPE_test': mape_test,
        'R^2_test': r2_test,
        'R^2_difference': r2_difference,
        'y_true_test': test_y,       # True values for the test set
        'y_pred_test': y_pred_test    # Predicted values for the test set
    }
    
    print(f"Results for {pair} -")
    print(f"Train - MAE: {mae_train}, MAPE: {mape_train}, R^2: {r2_train}")
    print(f"Test - MAE: {mae_test}, MAPE: {mape_test}, R^2: {r2_test}")
    print(f"Difference in R² (Train - Test): {r2_difference}\n")

# Display final results for training and test performance comparison
print("\nFinal Results for Each Currency Pair:")
for pair, metrics in results.items():
    print(f"{pair} - Train: MAE: {metrics['MAE_train']}, MAPE: {metrics['MAPE_train']}, R^2: {metrics['R^2_train']}")
    print(f"Test: MAE: {metrics['MAE_test']}, MAPE: {metrics['MAPE_test']}, R^2: {metrics['R^2_test']}")
    print(f"Difference in R² (Train - Test): {metrics['R^2_difference']}\n")
    print(f"True values (sample): {metrics['y_true_test'][:5].values}")
    print(f"Predicted values (sample): {metrics['y_pred_test'][:5].values}\n")


Training ARIMA model for currency pair: USD/INR


C:\Users\uzmap\AppData\Roaming\Python\Python312\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\uzmap\AppData\Roaming\Python\Python312\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\uzmap\AppData\Roaming\Python\Python312\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


KeyError: 'The `start` argument could not be matched to a location related to the index of the data.'

In [26]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score
import pandas as pd

# Convert Date column to datetime (if necessary)
data['Date'] = pd.to_datetime(data['Date'])

# List of unique currency pairs
currency_pairs = data['Currency Pair'].unique()

# Dictionary to store evaluation metrics for each currency pair
results = {}

# Loop over each currency pair
for pair in currency_pairs:
    print(f"Training Exponential Smoothing model for currency pair: {pair}")
    
    # Filter data for the current currency pair
    pair_data = data[data['Currency Pair'] == pair]
    
    # Sort by date to ensure proper time series order
    pair_data = pair_data.sort_values('Date')
    
    # Prepare data (use 'Date' as index, and 'Closing_price' as target variable)
    pair_data.set_index('Date', inplace=True)
    y = pair_data['Closing_price']
    
    # Split data into train and test sets (80-20 split)
    split_idx = int(len(pair_data) * 0.8)
    train_y = y.iloc[:split_idx]
    test_y = y.iloc[split_idx:]
    
    # Fit Exponential Smoothing model on the training set
    model = ExponentialSmoothing(train_y)  # adjust for your data's seasonality
    model_fit = model.fit()
    
    # Make predictions on the training set
    y_pred_train = model_fit.fittedvalues
    
    # Make predictions on the test set
    y_pred_test = model_fit.forecast(len(test_y))

    # Calculate evaluation metrics for both training and test sets
    mae_train = mean_absolute_error(train_y, y_pred_train)
    mape_train = mean_absolute_percentage_error(train_y, y_pred_train)
    r2_train = r2_score(train_y, y_pred_train)
    
    mae_test = mean_absolute_error(test_y, y_pred_test)
    mape_test = mean_absolute_percentage_error(test_y, y_pred_test)
    r2_test = r2_score(test_y, y_pred_test)
    
    # Calculate the difference in R² between training and test to check for overfitting
    r2_difference = r2_train - r2_test
    
    # Store the results for this currency pair
    results[pair] = {
        'MAE_train': mae_train,
        'MAPE_train': mape_train,
        'R^2_train': r2_train,
        'MAE_test': mae_test,
        'MAPE_test': mape_test,
        'R^2_test': r2_test,
        'R^2_difference': r2_difference,
        'y_true_test': test_y,       # True values for the test set
        'y_pred_test': y_pred_test    # Predicted values for the test set
    }

# Display final results for training and test performance comparison
print("\nFinal Results for Each Currency Pair:")
for pair, metrics in results.items():
    print(f"{pair} - Train: MAE: {metrics['MAE_train']}, MAPE: {metrics['MAPE_train']}, R^2: {metrics['R^2_train']}")
    print(f"Test: MAE: {metrics['MAE_test']}, MAPE: {metrics['MAPE_test']}, R^2: {metrics['R^2_test']}")
    print(f"Difference in R² (Train - Test): {metrics['R^2_difference']}\n")


Training Exponential Smoothing model for currency pair: USD/INR
Training Exponential Smoothing model for currency pair: EUR/USD
Training Exponential Smoothing model for currency pair: GBP/USD
Training Exponential Smoothing model for currency pair: USD/JPY
Training Exponential Smoothing model for currency pair: EUR/INR
Training Exponential Smoothing model for currency pair: JPY/INR
Training Exponential Smoothing model for currency pair: GBP/INR

Final Results for Each Currency Pair:
USD/INR - Train: MAE: 0.0011368392487149568, MAPE: 0.0025690282868317954, R^2: 0.9974903752905648
Test: MAE: 0.015526178142636066, MAPE: 0.029572967353344606, R^2: -11.677360103145238
Difference in R² (Train - Test): 12.674850478435804

EUR/USD - Train: MAE: 3.0032883798454437e-05, MAPE: 0.00778736854206581, R^2: 0.9716360712374088
Test: MAE: 0.00032227153142922414, MAPE: 0.08745413326759174, R^2: -8.800850608929341
Difference in R² (Train - Test): 9.77248668016675

GBP/USD - Train: MAE: 4.3869998379716616e-

C:\Users\uzmap\AppData\Roaming\Python\Python312\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\uzmap\AppData\Roaming\Python\Python312\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\uzmap\AppData\Roaming\Python\Python312\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\uzmap\AppData\Roaming\Python\Python312\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer ind